## Get events 

In [24]:
import wget

!rm *.ndk

url = 'http://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/jan76_dec13.ndk'
wget.download( url )

years = range(6,18)
months  = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
for year in years:
    for month in months:
        url='http://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/20%02d/%s%02d.ndk' % (year, month, year)
        #print(url)
        wget.download( url )

#### Parse ndk files to populate catalog 

In [40]:
import obspy
from obspy.geodetics.base import gps2dist_azimuth
from obspy.clients.syngine import Client
from obspy.taup import TauPyModel

cat = obspy.read_events('jan76_dec13.ndk')
cat = cat.filter("time > 1990-01-01T00:00",'time < 2006-01-01T00:00').filter("magnitude >= 5.8")

for year in years:
    for month in months:
        tmp = obspy.read_events('%s%02d.ndk' % (month, year))
        cat = cat + tmp.filter('magnitude >= 5.8')
        
print(cat)

/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/ndk/core.py:205: ObsPyNDKWarning: Could not parse event 13965 (faulty file?). Will be skipped. Lines of the event:
	PDE  1996/11/20 19:42:56.1  10.30  127.43  33.0 5.6 5.3 PHILIPPINE ISLANDS REGIO
	B112096E         B: 35   58  45 S:  0    0   0 M:  0    0   0 CMT: 1 BOXHD: 22.0
	CENTROID:      1.060.0  10.47 0.03  127.42 0.03  15.0  0.0 BDY  O-00000000000000
	24 -4.983 0.133 -0.092 0.153  5.075 0.173 -0.433 0.443  0.715 0.469 -2.898 0.141
	V10   6.430  4 246  -1.390  2 156  -5.050 86  45   5.740 338 41  -88 154 49  -92
Traceback (most recent call last):
  File "/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/ndk/core.py", line 197, in _read_ndk
    record = _read_lines(*lines)
  File "/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/ndk/core.py", line 522, in _read_lines
    map(float, line3[:58].split()[1:])
ValueError: could not convert string to float: '1.060.0'

  warnings.warn(msg, ObsPy

5599 Event(s) in Catalog:
1990-01-04T05:32:30.500000Z | -15.450, -173.170 | 6.48 Mwc
1990-01-05T18:27:00.200000Z | +18.540, -106.830 | 5.94 Mwc
...
2017-12-13T18:03:54.000000Z | -54.400,   +2.280 | 6.5 Mwc
2017-12-15T16:48:00.700000Z |  -7.910, +108.110 | 6.55 Mwc
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [41]:
from obspy.clients.fdsn.client import Client
from obspy.geodetics.base import gps2dist_azimuth
from sph_loc import sph_loc

import pandas as pd
import sqlite3

#Connect to Database
conn = sqlite3.connect("ss.db")
cur  = conn.cursor()

def save_dataframe(dataframe, if_exists = 'fail'):
    #dataframe.to_csv(file, index = False, mode = 'a', header = header)
    dataframe.to_sql("bouncepoints_full", conn, if_exists=if_exists,
                     dtype = {'ID': 'INTEGER',
                                'ID_Full:': "TEXT",
                                'Event': 'TEXT',
                                'Network': 'TEXT',
                                'Station':'TEXT',
                                'Range':'REAL',
                                'Azimuth':'REAL',
                                'BAzimuth':'REAL',
                                'EvtLat':'REAL',
                                'EvtLon':'REAL',
                                'EvtDep':'REAL',
                                'EvtMag':'REAL',
                                'EvtStr':'REAL',
                                'EvtDip':'REAL',
                                'EvtRak':'REAL',
                                'StaLat':'REAL',
                                'StaLon':'REAL',
                                'MidLat':'REAL',
                                'MidLon':'REAL',
                                'EvtTime': 'TEXT'}
                    )
    print('saved')
    
def init_dataframe():
    dataframe = pd.DataFrame(columns =['ID', 'ID_Full', 'Event', 'Network', 'Station', 'Range', 'Azimuth',
                                       'BAzimuth', 'EvtLat','EvtLon', 'EvtDep', 'EvtMag','EvtStr','EvtDip','EvtRak',
                                       'StaLat', 'StaLon', 'MidLat','MidLon', 'EvtTime'],
                            )
    
    return dataframe

df = init_dataframe()
    
save_dataframe(df, if_exists = 'replace')
    
irow = 0

client = Client('IRIS')

for evt in cat:

    etag = evt.event_descriptions[1].text
    
    etime = evt.preferred_origin().time
    elat  = evt.preferred_origin().latitude
    elon  = evt.preferred_origin().longitude
    edep  = evt.preferred_origin().depth/1000.0
    emag  = evt.preferred_magnitude().mag
    estr  = evt.focal_mechanisms[0].nodal_planes.nodal_plane_1.strike
    edip  = evt.focal_mechanisms[0].nodal_planes.nodal_plane_1.dip
    erak  = evt.focal_mechanisms[0].nodal_planes.nodal_plane_1.rake
    
    try:
        inv = client.get_stations(network='*', station='*', channel='LH?', level='station',
                                  startbefore = etime, endafter = etime+60*60,
                                  includerestricted = False, includeavailability=True)
    except:
        print('Request Error')
        continue
        
    for net in inv:
        for sta in net:
            if sta.restricted_status != 'open':
                continue
            
            slat = sta.latitude
            slon = sta.longitude
            
            delm, az, baz = gps2dist_azimuth(elat, elon, slat, slon)
            deldeg = delm/1000./111.11
            if deldeg > 55:
                mlat, mlon = sph_loc(elat,elon,deldeg/2.,az)
                
                if mlon > 180.:
                    mlon = mlon - 360.
                
                fullid = etag + '.' + net.code + '.' + sta.code
                df.loc[irow] = [irow, fullid, etag, net.code, sta.code, deldeg, az, baz, elat, elon, edep, emag, estr, edip, erak, slat, slon, mlat, mlon, str(etime)]
                irow += 1
                if irow % 1000 == 0:  #write and drop memory load
                    save_dataframe(df, 'append')
                    df = init_dataframe()

save_dataframe(df, 'append')

conn.close()

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save